In [1]:
from transformers import pipeline
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import time

2025-07-04 17:56:52.457547: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-04 17:56:55.941769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751644617.166559  504833 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751644617.563248  504833 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751644620.043730  504833 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

INFO 07-04 17:57:44 [__init__.py:244] Automatically detected platform cuda.


In [2]:
#model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
model_name = "google/gemma-3-27b-it"

In [3]:
start_time = time.time()

# Tell vLLM to use 2 GPUs for tensor parallelism.
# This automatically splits the model across the 2 A100s.
# The process takes about 30min!
llm = llm = LLM(model=model_name, tensor_parallel_size=2)

elapsed_time = (time.time() - start_time) / 60
print(f"LLM setup took {elapsed_time} min.")

# Load the tokenizer for the same model
tokenizer = AutoTokenizer.from_pretrained(model_name)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-3-27b-it.
401 Client Error. (Request ID: Root=1-6867fa04-0c00b3107041206f6a61c690;6a2d7773-ff46-48ed-972a-8dc988f1c4f8)

Cannot access gated repo for url https://huggingface.co/google/gemma-3-27b-it/resolve/main/config.json.
Access to model google/gemma-3-27b-it is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
# Load the tokenizer for the same model
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Define the conversation roles
messages = [
    {
        "role": "system",
        "content": "You are a friendly and helpful AI chatbot. Your goal is to assist the user with their questions in a clear and concise way."
    },
    {
        "role": "user",
        "content": "Bitte gib die Speaker stance für den folgenden Text aus den Labeln [Favour, Against, Neither] an. Target: \"der luxemburgischen Ratspräsidentschaft in\" (markiert durch <span> tag). Text: Die Einigung ist für uns deshalb mindestens so entscheidend wie für die anderen europäischen Staaten. Im Juni haben wir die Einigung unter <span>der luxemburgischen Ratspräsidentschaft in</span> Luxemburg schon einmal versucht. Ich sage voraus: Wenn wir am Ende dieses Jahres mit dem zweiten Versuch einer Einigung über den Finanzrahmen erneut scheitern würden, dann ginge davon ein verheerendes Signal für die Bürgerinnen und Bürger aus. Insbesondere darf nicht vergessen werden, dass sich ein Scheitern vor allem zulasten der neuen Mitgliedstaaten auswirken würde."
    }
]

# Applying the chat template to create the final prompt string.
#    - `tokenize=False` makes it return a string, which is what llm.generate expects.
#    - `add_generation_prompt=True` adds the special tokens to signal to the model that it's the assistant's turn to speak.
formatted_prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
# Single formatted prompt for model
prompts_to_generate = [formatted_prompt]


# sampling parameters for generation.
sampling_params = SamplingParams(
    temperature=0.8,
    top_p=0.95,
    max_tokens=1000 # Adjust max_tokens as needed
)

# Generate completions for all prompts in a batch using vllm

outputs = llm.generate(prompts_to_generate, sampling_params)

for output in outputs:
    # The 'prompt' here will be the long, formatted string with special tokens
    original_prompt_info = output.prompt
    generated_text = output.outputs[0].text
    
    print("--- Input to Model ---")
    print(f"{original_prompt_info!r}")
    print("\n--- Generated Output ---")
    print(f"{generated_text!r}")